In [ ]:
from numcosmo_py import Ncm, Nc
import sys
import math
import matplotlib.pyplot as plt
import numpy as np

from scipy.special import erfc

from numcosmo_py.plotting.tools import set_rc_params_article, latex_float

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

In [ ]:
set_rc_params_article(ncol=2)

In [ ]:
qgw = Nc.HICosmoQGW.new()
adiab = Nc.HIPertAdiab.new()

In [ ]:
qgw.props.xb = 1.0e35
qgw.props.w = 1.0e-18

In [ ]:
adiab.set_k(1.0e-1)
adiab.set_ti(-300.0)
adiab.set_tf(-1.0)
adiab.set_reltol(1.0e-11)
adiab.set_initial_condition_type(Ncm.CSQ1DInitialStateType.ADIABATIC4)
adiab.set_vacuum_max_time(-1.0e-15)
adiab.set_vacuum_reltol(1.0e-7)

Found, tAD = adiab.find_adiab_time_limit(
    qgw, adiab.get_ti(), adiab.get_vacuum_max_time(), adiab.get_vacuum_reltol()
)

print(f"Found: {Found}, tAD: {tAD}")

In [ ]:
adiab.prepare(qgw)

In [ ]:
ta, ts = adiab.get_time_array()

In [ ]:
Pzeta = [adiab.eval_powspec_zeta_at(qgw, t) for t in ta]
PPsi = [adiab.eval_powspec_Psi_at(qgw, t) for t in ta]
Pdrho = [adiab.eval_powspec_drho_at(qgw, t) for t in ta]

In [ ]:
fig = plt.figure()

plt.plot(ta, Pzeta, label="$P_\zeta$")
plt.plot(ta, PPsi, label="$P_\Psi$")
plt.plot(ta, Pdrho, label=r"$P_{\delta_\rho}$")

plt.hlines(2.0e-9, xmin=min(ta), xmax=max(ta))
plt.yscale("log")

plt.grid()
plt.legend()

In [ ]:
adiab.eval_cosmic_time(qgw, -170)
pass

In [ ]:
for k in np.geomspace(1.0e2, 1.0e17, 10):
    adiab.set_k(k)
    tau_min = -qgw.eval_tau_jeans(k)
    tau_max = -qgw.eval_tau_hubble(k)
    tau_l_a = np.linspace(tau_min, tau_max, 100)

    plt.plot(
        tau_l_a,
        [adiab.eval_delta_critial(qgw, tau) for tau in tau_l_a],
        label=rf"$k={latex_float(k)}$",
    )

plt.yscale("log")
plt.legend()
pass

In [ ]:
fig = plt.figure()

ka = np.geomspace(1.0e4, 2.0e17, 1000)
tau_jeans_a = [-qgw.eval_tau_jeans(k) for k in ka]
tau_hubble_a = [-qgw.eval_tau_hubble(k) for k in ka]
tau_a = np.linspace(min(tau_jeans_a), max(tau_hubble_a), 120)

plt.plot(ka, tau_jeans_a, label=rf"$\tau_\mathrm{{Jeans}}$")
plt.plot(ka, tau_hubble_a, label=rf"$\tau_\mathrm{{Hubble}}$")

plt.xscale("log")

plt.grid()
plt.legend()
pass

In [ ]:
ka = np.geomspace(1.0e2, 1.0e20, 1000)
tau_a = np.linspace(-120.0, -10.0, 500)
adiab.prepare_spectrum(qgw, ka, tau_a)

In [ ]:
ka = np.geomspace(1.0e2, 1.0e21, 1000)

In [ ]:
Pzetak = adiab.eval_powspec_zeta(qgw)
# Pzetak.set_kmax(1.0e21)
PPsik = adiab.eval_powspec_Psi(qgw)
# PPsik.set_kmax(1.0e21)
Pdrhok = adiab.eval_powspec_drho(qgw)
# Pdrhok.set_kmax(1.0e21)

In [ ]:
fig = plt.figure()

k = 1.0e15

ymin = 1.0e300
ymax = 1.0e-300

tau = -qgw.eval_tau_jeans(k)

Ps = [k**3 * 0.5 / np.pi**2 * Pzetak.eval(None, tau, k) for tau in tau_a]
plt.plot(tau_a, Ps, label=f"$P_\zeta({latex_float(tau)})$")

ymin = np.min(np.concatenate((Ps, [ymin])))
ymax = np.max(np.concatenate((Ps, [ymax])))

Ps = [k**3 * 0.5 / np.pi**2 * PPsik.eval(None, tau, k) for tau in tau_a]
plt.plot(tau_a, Ps, label=f"$P_\Psi({latex_float(tau)})$")

ymin = np.min(np.concatenate((Ps, [ymin])))
ymax = np.max(np.concatenate((Ps, [ymax])))

Ps = [k**3 * 0.5 / np.pi**2 * Pdrhok.eval(None, tau, k) for tau in tau_a]
plt.plot(tau_a, Ps, label=f"$P_\Psi({latex_float(tau)})$")

ymin = np.min(np.concatenate((Ps, [ymin])))
ymax = np.max(np.concatenate((Ps, [ymax])))

plt.vlines([-qgw.eval_tau_jeans(k), -qgw.eval_tau_hubble(k)], ymin=ymin, ymax=ymax)

plt.yscale("log")

pass

In [ ]:
fig = plt.figure()

ka = np.geomspace(1.0e2, 1.0e21, 1000)

tau_plot_a = tau_a[::40]
for tau in tau_plot_a:
    Ps = [k**3 * 0.5 / np.pi**2 * Pzetak.eval(None, tau, k) for k in ka]
    plt.plot(ka, Ps, label=f"$P_\zeta({latex_float(tau)})$")

plt.xscale("log")
plt.yscale("log")
plt.legend()
pass

In [ ]:
fig = plt.figure()

for tau in tau_plot_a:
    plt.plot(
        ka,
        [k**3 * 0.5 / np.pi**2 * PPsik.eval(None, tau, k) for k in ka],
        label=rf"$P_\Psi({latex_float(tau)})$",
    )

plt.xscale("log")
plt.yscale("log")
plt.legend()
pass

In [ ]:
fig = plt.figure()

for tau in tau_plot_a:
    plt.plot(
        ka,
        [k**3 * 0.5 / np.pi**2 * Pdrhok.eval(None, tau, k) for k in ka],
        label=rf"$P_{{\delta_\rho}}({latex_float(tau)})$",
    )

plt.xscale("log")
plt.yscale("log")
plt.legend()
plt.grid()
pass

In [ ]:
def compute_sigma_at_jeans(kJ):
    # Compute the time at which the wave-number k enters the Jeans scale
    tau_k_jeans = -qgw.eval_tau_jeans(kJ)
    # Compute the Hubble scale at that time
    kH = np.abs(qgw.eval_hubble(tau_k_jeans) / qgw.eval_x(tau_k_jeans))
    # Create a range of wave-numbers from the Jeans scale to the Hubble scale
    ka = np.geomspace(kH, kJ, 50)
    # Create a range of times from the time the wave-number k enters the Jeans scale to
    # the time it enters the Hubble scale
    tau_a = np.linspace(tau_k_jeans - 1.0e-1, tau_k_jeans + 1.0e-1, 6)
    # Prepare the spectrum for the range of wave-numbers and times
    adiab.prepare_spectrum(qgw, ka, tau_a)
    # Evaluate the power spectrum of the energy density perturbation at the time
    # the wave-number kJ enters the Jeans scale.
    Pdrhok = adiab.eval_powspec_drho(qgw)
    # Evaluate the variance of the energy density perturbation at the time
    # the wave-number kJ enters the Jeans scale.
    sigma_drho = Pdrhok.sigma_tophat_R(qgw, 1.0e-4, tau_k_jeans, 1.0 / kJ)

    # Compute the critial delta for
    adiab.set_k(kJ)
    delta_c = adiab.eval_delta_critial(qgw, tau_k_jeans)

    return delta_c, sigma_drho

In [ ]:
fig = plt.figure()

ka = np.geomspace(1.0e10, 1.0e15, 10)
sigma_delta = np.array([compute_sigma_at_jeans(k) for k in ka])

plt.plot(1.0 / ka, sigma_delta[:, 1], label=r"$\sigma(\tau_J, \lambda_J)$")
plt.plot(1.0 / ka, sigma_delta[:, 0], label=r"$\delta_c(\tau_J)$")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("$\lambda$")
plt.legend()
pass